# Optimization of Noise Generator

We want to find the right Hyperparameters for the Noise Generator.
For the Optimization, we will use the `Optuna` Framework

The correct Hyperparameters for the Noise will be found for:
- `GeFeU`
- `GEMU`
- `Gradient Ascent`

In [1]:
import optuna
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import random

import os
import sys

# Add the parent directory to sys.path
sys.path.append(os.path.join('..', 'src'))

import gefeu
import gemu
import mlp_dataclass
import metrics
from helper import load_models_dict

___
## GeFeU

In [2]:
import logging
import sys
import pickle
import optuna
import os

# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study1_name = "GenOptiGeFeU"  # Unique identifier of the study.
storage1_name = "sqlite:///{}.db".format("HP_Opti")

if os.path.exists("sampler_gefeu.pkl"):
    restored1_sampler = pickle.load(open("sampler_gefeu.pkl", "rb"))
    study_gefeu = optuna.create_study(study_name=study1_name, storage=storage1_name, load_if_exists=True, sampler=restored1_sampler)
else:
    study_gefeu = optuna.create_study(study_name=study1_name, storage=storage1_name, load_if_exists=True)

[I 2025-01-23 12:46:11,438] Using an existing study with name 'GenOptiGeFeU' instead of creating a new one.


Using an existing study with name 'GenOptiGeFeU' instead of creating a new one.


In [3]:
def objective(trial):

    opt_Epochs = trial.suggest_int('opt_Epochs', 1, 10)
    opt_Learning_Rate = trial.suggest_float('opt_Learning_Rate', 0.01, 0.3)
    opt_Batch_Size = trial.suggest_int('opt_Batch_Size', 32, 256)
    opt_N2R_Ratio = trial.suggest_float('opt_N2R_Ratio', 0.01, 20)
    opt_Regularization_term = trial.suggest_float('opt_Regularization_term', 0.01, 0.3)
    opt_Noise_Dim = trial.suggest_int('opt_Noise_Dim', 1, 512)
    opt_Impair_LR = trial.suggest_float('opt_Impair_LR', 0.01, 0.3)
    opt_Repair_LR = trial.suggest_float('opt_Repair_LR', 0.01, 0.3)

    n_layers = trial.suggest_int('n_layers', 1, 10)

    Layers = [1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024]
    Layers = Layers[:n_layers]

    random_int = random.randint(0, 29)
    train = load_models_dict(path="../data/models/mnist/all/test_ensemble")[random_int]
    
    unlearned = gefeu._main(
        model=train,
        dataset_name="mnist",
        t_Epochs = opt_Epochs,
        t_Learning_Rate = opt_Learning_Rate,
        t_Batch_Size = opt_Batch_Size,
        t_N2R_Ratio= opt_N2R_Ratio,
        t_Regularization_term = opt_Regularization_term,
        t_Layers = Layers,
        t_Noise_Dim = opt_Noise_Dim,
        t_Impair_LR=opt_Impair_LR,
        t_Repair_LR=opt_Repair_LR,
        logs=False,
        model_eval_logs=False,
    )

    valid_ds = mlp_dataclass.MNIST_CostumDataset(
        sample_mode="all",
        train=True,
        test=False,
        balanced=False,
        dataset_name="mnist",
        download=False,
    )  
    valid_dl = DataLoader(valid_ds, 256, shuffle=False)

    random_int = random.randint(0, 29)
    exact = load_models_dict(path="../data/models/mnist/except_erased/test_ensemble")[random_int]
    
    div = metrics.kl_divergence_between_models(
        model1 = exact,
        model2 = unlearned,
        data_loader = valid_dl,
    )

    return div

study_gefeu.optimize(objective, n_trials=1)

import pickle

# Save the sampler with pickle to be loaded later.
with open("sampler_gefeu.pkl", "wb") as fout:
    pickle.dump(study_gefeu.sampler, fout)

[I 2025-01-23 12:48:03,791] Trial 109 finished with value: 1.7550261066315023 and parameters: {'opt_Epochs': 7, 'opt_Learning_Rate': 0.24720191092541471, 'opt_Batch_Size': 82, 'opt_N2R_Ratio': 8.381157087245017, 'opt_Regularization_term': 0.06173199902189664, 'opt_Noise_Dim': 130, 'opt_Impair_LR': 0.19023417732467884, 'opt_Repair_LR': 0.02054467096826751, 'n_layers': 4}. Best is trial 99 with value: 0.48107353245958373.


Trial 109 finished with value: 1.7550261066315023 and parameters: {'opt_Epochs': 7, 'opt_Learning_Rate': 0.24720191092541471, 'opt_Batch_Size': 82, 'opt_N2R_Ratio': 8.381157087245017, 'opt_Regularization_term': 0.06173199902189664, 'opt_Noise_Dim': 130, 'opt_Impair_LR': 0.19023417732467884, 'opt_Repair_LR': 0.02054467096826751, 'n_layers': 4}. Best is trial 99 with value: 0.48107353245958373.


In [4]:
u = load_models_dict(path="../data/models/mnist/all/test_ensemble")[1]
z = load_models_dict(path="../data/models/mnist/except_erased/test_ensemble")[1]
valid_ds = mlp_dataclass.MNIST_CostumDataset(
        sample_mode="all",
        train=False,
        test=True,
        balanced=False,
        dataset_name="mnist",
        download=False,
    )  
valid_dl = DataLoader(valid_ds, 56, shuffle=False)

In [5]:
metrics.kl_divergence_between_models(z, u, valid_dl, device='cpu')

0.04423580976598508

## Standard Values

In [6]:
import gefeu

standard_model_gefeu = gefeu._main(
    model=u,
    dataset_name="mnist",
    t_Epochs=study_gefeu.best_params["opt_Epochs"],
    t_Batch_Size=study_gefeu.best_params["opt_Batch_Size"],
    t_Learning_Rate=study_gefeu.best_params["opt_Learning_Rate"],
    t_N2R_Ratio=study_gefeu.best_params["opt_N2R_Ratio"],
    t_Regularization_term=study_gefeu.best_params["opt_Regularization_term"],
    t_Layers=[1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024][:study_gefeu.best_params["n_layers"]],
    t_Noise_Dim=study_gefeu.best_params["opt_Noise_Dim"],
    t_Impair_LR=study_gefeu.best_params["opt_Impair_LR"],
    t_Repair_LR=study_gefeu.best_params["opt_Repair_LR"],
    logs=True,
    model_eval_logs=False,
)

Baseline Performance
Time: 0.10838794708251953
Epoch: 1, Loss: 21.482921600341797
Epoch: 2, Loss: -6.196435928344727
Epoch: 3, Loss: -6.265256881713867
Epoch: 4, Loss: -6.27152681350708
Epoch: 5, Loss: -6.263731002807617
Epoch: 6, Loss: -6.237681865692139
Epoch: 7, Loss: -6.208515644073486
Ending Noise Maximazation Phase
Time: 69.6134819984436
______
Starting Impairing Phase
Time: 69.7035300731659
Train loss 1: 8.521982576479012
Ending Impairing Phase
Time: 103.49837398529053
Train loss 1: 4.078282943926262
Ending Repairing Phase
Time: 112.76995205879211
______


___
## GEMU

In [7]:
import logging
import sys
import pickle
import optuna
import os


# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study2_name = "GenOptiGEMU"  # Unique identifier of the study.
storage2_name = "sqlite:///{}.db".format("HP_Opti")

if os.path.exists("sampler_gemu.pkl"):
    restored2_sampler = pickle.load(open("sampler_gemu.pkl", "rb"))
    study_gemu = optuna.create_study(study_name=study2_name, storage=storage2_name, load_if_exists=True, sampler=restored2_sampler)
else:
    study_gemu = optuna.create_study(study_name=study2_name, storage=storage2_name, load_if_exists=True)

[I 2025-01-23 12:50:07,317] A new study created in RDB with name: GenOptiGEMU


A new study created in RDB with name: GenOptiGEMU
A new study created in RDB with name: GenOptiGEMU


In [8]:
def objective(trial):

    opt_Epochs = trial.suggest_int('opt_Epochs', 1, 10)
    opt_Learning_Rate = trial.suggest_float('opt_Learning_Rate', 0.01, 0.3)
    opt_Batch_Size = trial.suggest_int('opt_Batch_Size', 32, 256)
    opt_N2R_Ratio = trial.suggest_float('opt_N2R_Ratio', 0.01, 20)
    opt_Regularization_term = trial.suggest_float('opt_Regularization_term', 0.01, 0.3)
    opt_Noise_Dim = trial.suggest_int('opt_Noise_Dim', 1, 512)
    opt_Impair_LR = trial.suggest_float('opt_Impair_LR', 0.01, 0.3)
    opt_Repair_LR = trial.suggest_float('opt_Repair_LR', 0.01, 0.3)

    n_layers = trial.suggest_int('n_layers', 1, 10)

    Layers = [1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024]
    Layers = Layers[:n_layers]

    random_int = random.randint(0, 29)
    train = load_models_dict(path="../data/models/mnist/all/test_ensemble")[random_int]

    unlearned = gemu._main(
        model=train,
        dataset_name="mnist",
        t_Epochs = opt_Epochs,
        t_Learning_Rate = opt_Learning_Rate,
        t_Batch_Size = opt_Batch_Size,
        t_N2R_Ratio= opt_N2R_Ratio,
        t_Regularization_term = opt_Regularization_term,
        t_Layers = Layers,
        t_Noise_Dim = opt_Noise_Dim,
        t_Impair_LR=opt_Impair_LR,
        t_Repair_LR=opt_Repair_LR,
        logs=False,
        model_eval_logs=False,
    )

    valid_ds = mlp_dataclass.MNIST_CostumDataset(
        sample_mode="all",
        train=True,
        test=False,
        balanced=False,
        dataset_name="mnist",
        download=False,
    )  
    valid_dl = DataLoader(valid_ds, 256, shuffle=False)

    random_int = random.randint(0, 29)
    exact = load_models_dict(path="../data/models/mnist/except_erased/test_ensemble")[random_int]
    
    div = metrics.kl_divergence_between_models(
        model1 = exact,
        model2 = unlearned,
        data_loader = valid_dl,
    )

    return div

study_gemu.optimize(objective, n_trials=1)

import pickle

# Save the sampler with pickle to be loaded later.
with open("sampler_gemu.pkl", "wb") as fout:
    pickle.dump(study_gemu.sampler, fout)

[I 2025-01-23 12:51:49,512] Trial 0 finished with value: 1.6851281084912881 and parameters: {'opt_Epochs': 3, 'opt_Learning_Rate': 0.28267545038269026, 'opt_Batch_Size': 154, 'opt_N2R_Ratio': 7.160401664730708, 'opt_Regularization_term': 0.15591929383665737, 'opt_Noise_Dim': 439, 'opt_Impair_LR': 0.2060646047983842, 'opt_Repair_LR': 0.0767537599106409, 'n_layers': 10}. Best is trial 0 with value: 1.6851281084912881.


Trial 0 finished with value: 1.6851281084912881 and parameters: {'opt_Epochs': 3, 'opt_Learning_Rate': 0.28267545038269026, 'opt_Batch_Size': 154, 'opt_N2R_Ratio': 7.160401664730708, 'opt_Regularization_term': 0.15591929383665737, 'opt_Noise_Dim': 439, 'opt_Impair_LR': 0.2060646047983842, 'opt_Repair_LR': 0.0767537599106409, 'n_layers': 10}. Best is trial 0 with value: 1.6851281084912881.
Trial 0 finished with value: 1.6851281084912881 and parameters: {'opt_Epochs': 3, 'opt_Learning_Rate': 0.28267545038269026, 'opt_Batch_Size': 154, 'opt_N2R_Ratio': 7.160401664730708, 'opt_Regularization_term': 0.15591929383665737, 'opt_Noise_Dim': 439, 'opt_Impair_LR': 0.2060646047983842, 'opt_Repair_LR': 0.0767537599106409, 'n_layers': 10}. Best is trial 0 with value: 1.6851281084912881.


[I 2025-01-23 12:55:20,738] Trial 1 finished with value: 1.7586434521573653 and parameters: {'opt_Epochs': 7, 'opt_Learning_Rate': 0.18381519850352634, 'opt_Batch_Size': 115, 'opt_N2R_Ratio': 15.890273152000393, 'opt_Regularization_term': 0.1971893482376725, 'opt_Noise_Dim': 221, 'opt_Impair_LR': 0.27007237005678203, 'opt_Repair_LR': 0.19835721749113414, 'n_layers': 2}. Best is trial 0 with value: 1.6851281084912881.


Trial 1 finished with value: 1.7586434521573653 and parameters: {'opt_Epochs': 7, 'opt_Learning_Rate': 0.18381519850352634, 'opt_Batch_Size': 115, 'opt_N2R_Ratio': 15.890273152000393, 'opt_Regularization_term': 0.1971893482376725, 'opt_Noise_Dim': 221, 'opt_Impair_LR': 0.27007237005678203, 'opt_Repair_LR': 0.19835721749113414, 'n_layers': 2}. Best is trial 0 with value: 1.6851281084912881.
Trial 1 finished with value: 1.7586434521573653 and parameters: {'opt_Epochs': 7, 'opt_Learning_Rate': 0.18381519850352634, 'opt_Batch_Size': 115, 'opt_N2R_Ratio': 15.890273152000393, 'opt_Regularization_term': 0.1971893482376725, 'opt_Noise_Dim': 221, 'opt_Impair_LR': 0.27007237005678203, 'opt_Repair_LR': 0.19835721749113414, 'n_layers': 2}. Best is trial 0 with value: 1.6851281084912881.


[I 2025-01-23 12:57:00,213] Trial 2 finished with value: 1.705736121725528 and parameters: {'opt_Epochs': 1, 'opt_Learning_Rate': 0.1412013496570691, 'opt_Batch_Size': 41, 'opt_N2R_Ratio': 11.974100708928859, 'opt_Regularization_term': 0.04124307848727003, 'opt_Noise_Dim': 366, 'opt_Impair_LR': 0.24790768578222638, 'opt_Repair_LR': 0.25678056511618547, 'n_layers': 2}. Best is trial 0 with value: 1.6851281084912881.


...

Trial 97 finished with value: 0.5865748268492676 and parameters: {'opt_Epochs': 5, 'opt_Learning_Rate': 0.028416774667475088, 'opt_Batch_Size': 54, 'opt_N2R_Ratio': 6.346598946565139, 'opt_Regularization_term': 0.2728192723234709, 'opt_Noise_Dim': 46, 'opt_Impair_LR': 0.06000714563619784, 'opt_Repair_LR': 0.023481234080966203, 'n_layers': 8}. Best is trial 53 with value: 0.4791380806172145.
Trial 97 finished with value: 0.5865748268492676 and parameters: {'opt_Epochs': 5, 'opt_Learning_Rate': 0.028416774667475088, 'opt_Batch_Size': 54, 'opt_N2R_Ratio': 6.346598946565139, 'opt_Regularization_term': 0.2728192723234709, 'opt_Noise_Dim': 46, 'opt_Impair_LR': 0.06000714563619784, 'opt_Repair_LR': 0.023481234080966203, 'n_layers': 8}. Best is trial 53 with value: 0.4791380806172145.


[I 2025-01-23 16:05:28,111] Trial 98 finished with value: 0.9607749982083096 and parameters: {'opt_Epochs': 6, 'opt_Learning_Rate': 0.035402532815687275, 'opt_Batch_Size': 59, 'opt_N2R_Ratio': 8.602838319782203, 'opt_Regularization_term': 0.2858224906863787, 'opt_Noise_Dim': 339, 'opt_Impair_LR': 0.07383748058504148, 'opt_Repair_LR': 0.04201784082043945, 'n_layers': 1}. Best is trial 53 with value: 0.4791380806172145.


Trial 98 finished with value: 0.9607749982083096 and parameters: {'opt_Epochs': 6, 'opt_Learning_Rate': 0.035402532815687275, 'opt_Batch_Size': 59, 'opt_N2R_Ratio': 8.602838319782203, 'opt_Regularization_term': 0.2858224906863787, 'opt_Noise_Dim': 339, 'opt_Impair_LR': 0.07383748058504148, 'opt_Repair_LR': 0.04201784082043945, 'n_layers': 1}. Best is trial 53 with value: 0.4791380806172145.
Trial 98 finished with value: 0.9607749982083096 and parameters: {'opt_Epochs': 6, 'opt_Learning_Rate': 0.035402532815687275, 'opt_Batch_Size': 59, 'opt_N2R_Ratio': 8.602838319782203, 'opt_Regularization_term': 0.2858224906863787, 'opt_Noise_Dim': 339, 'opt_Impair_LR': 0.07383748058504148, 'opt_Repair_LR': 0.04201784082043945, 'n_layers': 1}. Best is trial 53 with value: 0.4791380806172145.


[I 2025-01-23 16:08:15,755] Trial 99 finished with value: 0.5689076624018077 and parameters: {'opt_Epochs': 7, 'opt_Learning_Rate': 0.023794465448808297, 'opt_Batch_Size': 42, 'opt_N2R_Ratio': 17.53411272590383, 'opt_Regularization_term': 0.2607006157119911, 'opt_Noise_Dim': 61, 'opt_Impair_LR': 0.05560317478250651, 'opt_Repair_LR': 0.033403925578916645, 'n_layers': 2}. Best is trial 53 with value: 0.4791380806172145.


Trial 99 finished with value: 0.5689076624018077 and parameters: {'opt_Epochs': 7, 'opt_Learning_Rate': 0.023794465448808297, 'opt_Batch_Size': 42, 'opt_N2R_Ratio': 17.53411272590383, 'opt_Regularization_term': 0.2607006157119911, 'opt_Noise_Dim': 61, 'opt_Impair_LR': 0.05560317478250651, 'opt_Repair_LR': 0.033403925578916645, 'n_layers': 2}. Best is trial 53 with value: 0.4791380806172145.
Trial 99 finished with value: 0.5689076624018077 and parameters: {'opt_Epochs': 7, 'opt_Learning_Rate': 0.023794465448808297, 'opt_Batch_Size': 42, 'opt_N2R_Ratio': 17.53411272590383, 'opt_Regularization_term': 0.2607006157119911, 'opt_Noise_Dim': 61, 'opt_Impair_LR': 0.05560317478250651, 'opt_Repair_LR': 0.033403925578916645, 'n_layers': 2}. Best is trial 53 with value: 0.4791380806172145.


## Standard Values

In [9]:
import gemu

standard_model_gemu = gemu._main(
    model=u,
    dataset_name="mnist",
    t_Epochs=study_gemu.best_params["opt_Epochs"],
    t_Batch_Size=study_gemu.best_params["opt_Batch_Size"],
    t_Learning_Rate=study_gemu.best_params["opt_Learning_Rate"],
    t_N2R_Ratio=study_gemu.best_params["opt_N2R_Ratio"],
    t_Regularization_term=study_gemu.best_params["opt_Regularization_term"],
    t_Layers=[1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024][:study_gemu.best_params["n_layers"]],
    t_Noise_Dim=study_gemu.best_params["opt_Noise_Dim"],
    t_Impair_LR=study_gemu.best_params["opt_Impair_LR"],
    t_Repair_LR=study_gemu.best_params["opt_Repair_LR"],
    logs=True,
    model_eval_logs=False,
)

Baseline Performance
Time: 0.10074710845947266
Epoch: 1, Loss: -88000.8671875
Epoch: 2, Loss: -90298.9375
Epoch: 3, Loss: -90210.203125
Epoch: 4, Loss: -90174.6015625
Epoch: 5, Loss: -90443.625
Epoch: 6, Loss: -90286.9140625
Ending NoiseGenerator Maximazation Phase
Time: 43.48147010803223
______
Starting Impairing Phase
Time: 43.57169198989868
Train loss 1: 35.66211067249977
Ending Impairing Phase
Time: 92.891606092453
Train loss 1: 5.133904508031208
Ending Repairing Phase
Time: 115.92272400856018
______


___
## Gradient Ascent

In [10]:
import logging
import sys
import pickle
import optuna
import os


# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study3_name = "OptiGA"  # Unique identifier of the study.
storage3_name = "sqlite:///{}.db".format("HP_Opti")

if os.path.exists("sampler_ga.pkl"):
    restored3_sampler = pickle.load(open("sampler_ga.pkl", "rb"))
    study_ga = optuna.create_study(study_name=study3_name, storage=storage3_name, load_if_exists=True, sampler=restored3_sampler)
else:
    study_ga = optuna.create_study(study_name=study3_name, storage=storage3_name, load_if_exists=True,)

[I 2025-01-23 16:10:11,758] A new study created in RDB with name: OptiGA


A new study created in RDB with name: OptiGA
A new study created in RDB with name: OptiGA
A new study created in RDB with name: OptiGA


In [13]:
from unlearning import SimpleGradientAscent

def objective(trial):

    opt_Epochs = trial.suggest_int('opt_Epochs', 1, 10)
    opt_Learning_Rate = trial.suggest_float('opt_Learning_Rate', 0, 1)
    opt_Batch_Size = trial.suggest_int('opt_Batch_Size', 1, 256)

    random_int = random.randint(0, 29)
    train = load_models_dict(path="../data/models/mnist/all/test_ensemble")[random_int]

    forget_ds = mlp_dataclass.MNIST_CostumDataset(
        sample_mode="only_erased",
        train=True,
        test=False,
        balanced=False,
        dataset_name="mnist",
        download=False,
    )
    forget_dl = valid_dl = DataLoader(forget_ds, opt_Batch_Size, shuffle=False)

    unlearned = SimpleGradientAscent(
        model=train,
        unlearned_data=forget_dl,
        dataset_name="mnist",
        t_LR = opt_Learning_Rate,
        t_Epochs = opt_Epochs,
        ).unlearn()

    valid_ds = mlp_dataclass.MNIST_CostumDataset(
        sample_mode="all",
        train=True,
        test=False,
        balanced=False,
        dataset_name="mnist",
        download=False,
    )  
    valid_dl = DataLoader(valid_ds, 256, shuffle=False)

    random_int = random.randint(0, 29)
    exact = load_models_dict(path="../data/models/mnist/except_erased/test_ensemble")[random_int]
    
    div = metrics.kl_divergence_between_models(
        model1 = exact,
        model2 = unlearned,
        data_loader = valid_dl,
    )

    return div

study_ga.optimize(objective, n_trials=1)

import pickle

# Save the sampler with pickle to be loaded later.
with open("sampler_ga.pkl", "wb") as fout:
    pickle.dump(study_ga.sampler, fout)

[I 2025-01-23 17:12:22,270] Trial 100 finished with value: 3.1702903947931644 and parameters: {'opt_Epochs': 1, 'opt_Learning_Rate': 0.8385184769281354, 'opt_Batch_Size': 180}. Best is trial 99 with value: 3.121353491823722.


Trial 100 finished with value: 3.1702903947931644 and parameters: {'opt_Epochs': 1, 'opt_Learning_Rate': 0.8385184769281354, 'opt_Batch_Size': 180}. Best is trial 99 with value: 3.121353491823722.
Trial 100 finished with value: 3.1702903947931644 and parameters: {'opt_Epochs': 1, 'opt_Learning_Rate': 0.8385184769281354, 'opt_Batch_Size': 180}. Best is trial 99 with value: 3.121353491823722.
Trial 100 finished with value: 3.1702903947931644 and parameters: {'opt_Epochs': 1, 'opt_Learning_Rate': 0.8385184769281354, 'opt_Batch_Size': 180}. Best is trial 99 with value: 3.121353491823722.


[I 2025-01-23 17:12:31,924] Trial 101 finished with value: 3.4108012980603144 and parameters: {'opt_Epochs': 1, 'opt_Learning_Rate': 0.8026956802681777, 'opt_Batch_Size': 182}. Best is trial 99 with value: 3.121353491823722.


Trial 101 finished with value: 3.4108012980603144 and parameters: {'opt_Epochs': 1, 'opt_Learning_Rate': 0.8026956802681777, 'opt_Batch_Size': 182}. Best is trial 99 with value: 3.121353491823722.
Trial 101 finished with value: 3.4108012980603144 and parameters: {'opt_Epochs': 1, 'opt_Learning_Rate': 0.8026956802681777, 'opt_Batch_Size': 182}. Best is trial 99 with value: 3.121353491823722.
Trial 101 finished with value: 3.4108012980603144 and parameters: {'opt_Epochs': 1, 'opt_Learning_Rate': 0.8026956802681777, 'opt_Batch_Size': 182}. Best is trial 99 with value: 3.121353491823722.


[I 2025-01-23 17:12:41,827] Trial 102 finished with value: 4.116389963474678 and parameters: {'opt_Epochs': 1, 'opt_Learning_Rate': 0.8406420431899663, 'opt_Batch_Size': 191}. Best is trial 99 with value: 3.121353491823722.


Trial 102 finished with value: 4.116389963474678 and parameters: {'opt_Epochs': 1, 'opt_Learning_Rate': 0.8406420431899663, 'opt_Batch_Size': 191}. Best is trial 99 with value: 3.121353491823722.
Trial 102 finished with value: 4.116389963474678 and parameters: {'opt_Epochs': 1, 'opt_Learning_Rate': 0.8406420431899663, 'opt_Batch_Size': 191}. Best is trial 99 with value: 3.121353491823722.
Trial 102 finished with value: 4.116389963474678 and parameters: {'opt_Epochs': 1, 'opt_Learning_Rate': 0.8406420431899663, 'opt_Batch_Size': 191}. Best is trial 99 with value: 3.121353491823722.


[I 2025-01-23 17:12:51,653] Trial 103 finished with value: 3.926605093225519 and parameters: {'opt_Epochs': 1, 'opt_Learning_Rate': 0.9949145952388141, 'opt_Batch_Size': 187}. Best is trial 99 with value: 3.121353491823722.


Trial 103 finished with value: 3.926605093225519 and parameters: {'opt_Epochs': 1, 'opt_Learning_Rate': 0.9949145952388141, 'opt_Batch_Size': 187}. Best is trial 99 with value: 3.121353491823722.
Trial 103 finished with value: 3.926605093225519 and parameters: {'opt_Epochs': 1, 'opt_Learning_Rate': 0.9949145952388141, 'opt_Batch_Size': 187}. Best is trial 99 with value: 3.121353491823722.
Trial 103 finished with value: 3.926605093225519 and parameters: {'opt_Epochs': 1, 'opt_Learning_Rate': 0.9949145952388141, 'opt_Batch_Size': 187}. Best is trial 99 with value: 3.121353491823722.


[I 2025-01-23 17:13:06,874] Trial 104 finished with value: 3.959106477524371 and parameters: {'opt_Epochs': 8, 'opt_Learning_Rate': 0.8976092893894684, 'opt_Batch_Size': 163}. Best is trial 99 with value: 3.121353491823722.


Trial 104 finished with value: 3.959106477524371 and parameters: {'opt_Epochs': 8, 'opt_Learning_Rate': 0.8976092893894684, 'opt_Batch_Size': 163}. Best is trial 99 with value: 3.121353491823722.
Trial 104 finished with value: 3.959106477524371 and parameters: {'opt_Epochs': 8, 'opt_Learning_Rate': 0.8976092893894684, 'opt_Batch_Size': 163}. Best is trial 99 with value: 3.121353491823722.
Trial 104 finished with value: 3.959106477524371 and parameters: {'opt_Epochs': 8, 'opt_Learning_Rate': 0.8976092893894684, 'opt_Batch_Size': 163}. Best is trial 99 with value: 3.121353491823722.


## Standard Values

In [12]:
forget_ds = mlp_dataclass.MNIST_CostumDataset(
    sample_mode="only_erased",
    train=True,
    test=False,
    balanced=False,
    dataset_name="mnist",
    download=False,
)
forget_dl = DataLoader(forget_ds, batch_size=study_ga.best_params["opt_Batch_Size"], shuffle=False)

unlearned_model_ga = SimpleGradientAscent(
    model=u,
    unlearned_data=forget_dl,
    dataset_name="mnist",
    t_LR = study_ga.best_params["opt_Learning_Rate"],
    t_Epochs = study_ga.best_params["opt_Epochs"],
    ).unlearn()

___